In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import *
from matplotlib import pyplot as plt

NUM_TRAIN = 49000
cifar_dict = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog','frog','horse','ship','truck' ]
# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914), (0.2023))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10(
    './cs231n/datasets',
    train=True, download=True,
    transform=transform
)
loader_train = DataLoader(
    cifar10_train,
    batch_size=64,
    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN))
)
cifar10_val = dset.CIFAR10(
    './cs231n/datasets',
    train=True, download=True,
    transform=transform
)
loader_val = DataLoader(
    cifar10_val, batch_size=64, 
    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000))
)

cifar10_test = dset.CIFAR10(
    './cs231n/datasets',
    train=False, download=True,
    transform=transform
)
loader_test = DataLoader(cifar10_test, batch_size=64)

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def train_part34(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
def predict(loader, model):
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            grid = np.concatenate([np.swapaxes(np.swapaxes(x[i].cpu().reshape(3,32,32),0,1),1,2) for i in range(10)], axis = 1)
            imshow(grid)
            final_pred = torch.argmax(scores,axis = 1)[:10].cpu()
            for i in final_pred:
              print(cifar_dict[i], end = " ")
            print()
            plt.show()
#fix this
class Dblock(nn.Module):
    def __init__(self, in_planes, planes):
        super(Dblock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, padding =1)
        self.conv2 = nn.Conv2d(planes, planes, 3, padding =1)
        self.conv3 = nn.Conv2d(planes, planes, 3, padding =1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.bn3 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if (in_planes != planes):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes,planes,3,padding = 1),
                nn.BatchNorm2d(planes)        
            )
    def forward(self,x):
        out1 = self.conv1(F.relu(self.bn1(x)))
        out2 = self.conv2(F.relu(self.bn1(self.shortcut(out1) + self.shortcut(x))))
        out3 = self.conv3(F.relu(self.bn1(self.shortcut(out2) + self.shortcut(out1) + self.shortcut(x))))
        return out3
class Block(nn.Module):
    def __init__(self, in_planes, planes):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, padding =1)
        self.conv2 = nn.Conv2d(planes, planes, 3, padding =1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if (in_planes != planes):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes,planes,3,padding = 1),
                nn.BatchNorm2d(planes)
            )
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)) + self.shortcut(x))
        return out
class ResNet(nn.Module):
    def __init__(self,in_channels, hidden_channels, num_classes):
        super(ResNet, self).__init__()
        self.conv = nn.Conv2d(in_channels, hidden_channels[0], 3, padding = 1)
        self.bn = nn.BatchNorm2d(hidden_channels[0])
        self.res1 = Block(hidden_channels[0], hidden_channels[1])
        self.res2 = Block(hidden_channels[1], hidden_channels[2])
        self.res3 = Block(hidden_channels[2], hidden_channels[3])
        self.maxpool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(hidden_channels[3]*16*16, num_classes)
    def forward(self,x):
        out = F.relu(self.bn(self.conv(x)))
        out = self.res1(out)
        out = self.res2(out)
        out = self.res3(out)
        out = self.maxpool(out)
        out = self.fc(flatten(out))
        return out

def train(model, optimizer, loader_train, loader_val, print_every = 100,epochs=1):
    model = model.to(device=device)
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x = x.to(device=device, dtype = dtype)
            y = y.to(device=device, dtype = torch.long)
            scores = model(x)
            loss = F.cross_entropy(scores,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f'%(t,loss.item()))
                check(loader_val,model)
                print()
def check(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct+= (preds==y).sum()
            num_samples+= preds.size(0)
        acc = float(num_correct)/num_samples
        print('Got %d / %d correct (%.2f)'%(num_correct, num_samples, 100*acc))
class block(nn.Module): 
    def __init__(self, in_planes, planes):
        super(block, self).__init__()
        #conv-bn-shortcut
        self.conv1 = nn.Conv2d(in_planes,planes,3, padding=1)
        self.conv2 = nn.Conv2d(planes,planes,3, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if planes != in_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes,planes,3, padding=1),
                nn.BatchNorm2d(planes)
                ) 
    def forward(self,x):
        out = F.relu((self.bn1(self.conv1(x))))
        out = F.relu((self.bn2(self.conv2(out)))+ self.shortcut(x))
        return out
class resNet(nn.Module):
    def __init__(self,in_channels,hidden_channels,num_classes, width = 32, height = 32):
        super(resNet,self).__init__()
        self.conv = nn.Conv2d(in_channels,hidden_channels[0], 3, padding=1)
        self.bn = nn.BatchNorm2d(hidden_channels[0])
        self.res1 = block(hidden_channels[0],hidden_channels[1])
        self.res2 = block(hidden_channels[1],hidden_channels[2])
        self.res3 = block(hidden_channels[2],hidden_channels[3])
        self.maxpool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(hidden_channels[3]*int(width/2)*int(height/2), num_classes)
    def forward(self,x):
        out = F.relu(self.bn(self.conv(x)))
        out = self.res1(out)
        out = self.res2(out)
        out = self.res3(out)
        out = self.maxpool(out)
        out = self.fc(flatten(out))
        return out


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [ ]:

hidden_layers = [16,32,64,128]
model = resNet(3, hidden_layers,10,32,32)
optimizer = optim.Adam(model.parameters(),lr = 2e-4)
train(model,optimizer,loader_train, loader_val, epochs = 20)


In [ ]:
best_model = model
check(loader_test, best_model)

Got 7246 / 10000 correct (72.46)


In [ ]:
best_model = model
predict(loader_test, best_model)